In [ ]:
%matplotlib inline
import numpy as np
import scipy.special as sc
from scipy import stats
import scipy.linalg as scalg
import matplotlib.pyplot as plt
import pdb
from deprojectVis import deproject_vis
from scipy.linalg import cho_factor, cho_solve

In [ ]:
#Squared exponential kernel
def exp2kernel(i1, i2, ibins=None, a=None, l=None):
    ri = ibins[i1]
    rj = ibins[i2]
    tol = 1e-6
    nugget = np.zeros_like(ri)
    nugget[(ri-rj)<tol] = 1e-16 #AD HOC!        
    return a*a * np.exp(-((ri - rj)**2.)/(2.*l*l))+nugget

#Covariance matrix for intensities
def calccovar(binsin, ain, lin):
    '''
    Calculate the covariance matrix using a squared exponential kernel
    a: normalization amplitude
    l: correlation length
    '''
    nbins = binsin.shape[0]
    cov = np.fromfunction(exp2kernel,(nbins,nbins), ibins=binsin, a=ain, l=lin, dtype = np.int)
    return cov


In [ ]:
##1 - Data setup
##We have Ndata data visibilities (D), with covariance matrix Sigma

#1a. Read in visibility data
visfilename = 'DATA/fullA_nf_discrete.vis.npz' ##Set _nf_discrete
datfile = np.load(visfilename)
Dorig = datfile['Vis']
uorig = datfile['u']
vorig = datfile['v']
rhoorig = np.sqrt(uorig**2. + vorig**2.)
Dwgtorig = datfile['Wgt']
#Sigmainv = np.diag(Dwgtorig) #a little unnecessary

#1b. Deproject (and optionally bin) the visibilities so that they're face-on
incl = 50. #deg
PA = 70. #deg
offx = -0.3 #arcsec
offy = -0.2 #arcsec
#visbins = np.arange(1., np.amax(rhoorig)/1000, 10) #Visibility bins
nvisbins = 100. ##Set
if (nvisbins>1):
   #visbins = np.linspace(np.amin(rhoorig)/1000., np.amax(rhoorig)/1000., nvisbins)
   #!!MAY NEED CHANGING!!
   visbins = stats.mstats.mquantiles(rhoorig/1000, np.arange(nvisbins)/nvisbins) #Changing the bins to have roughly even numbers of visibilities in each        
   rhodeproj, Ddeproj, sigdeproj = deproject_vis([uorig, vorig, Dorig, Dwgtorig], visbins, incl, PA, offx, offy, errtype='scat')
else:
   rhodeproj, Ddeproj, sigdeproj = deproject_vis([uorig, vorig, Dorig, Dwgtorig], incl=incl, PA=PA, offx=offx, offy=offy)

#1c. Set data variables (currently only using real parts)
arcsec = 180./np.pi*3600.
D = Ddeproj.real
rho = rhodeproj/arcsec #units of 1/arcsec
Sigmainv = np.diag(1./np.square(sigdeproj.real))
Ndata = D.size
print 'Number of vis is', Ndata, np.shape(rho), np.shape(Sigmainv)

In [ ]:
##2 - Model Setup
##We have a model visibility (M) which uses Nrings annuli

#Select model annuli radii in arcsec
rmin = 0.01/140.
rmax = 1.1
Nrings = 40
radii = np.linspace(rmin, rmax, num=Nrings+1) #Currently does NOT use rin
rleft = radii[:-1]
rright = radii[1:]
rcenter = (rleft+rright)/2.

#M=Xw, where X is only a function of the bins and baselines
X = np.empty([Ndata,Nrings])
for j in range(Ndata):
    for i in range(Nrings):
        X[j][i] = 1./rho[j]*(rright[i]*sc.j1(rho[j]*2.*np.pi*rright[i]) - rleft[i]*sc.j1(rho[j]*2.*np.pi*rleft[i]))

In [ ]:
##3 - Compute linear algebra

#Calculate uniform prior mean and covariance matrix
#The mean of the distribution with a uniform prior is wu, with covariance Cu
Cu = np.dot(np.dot(X.T, Sigmainv), X)
#Alternate method without inverse
wu = np.linalg.solve(Cu, np.dot(np.dot(X.T, Sigmainv), D)) #better than wu0

In [ ]:
#Calculate the mean to use for GP kernel (for now it's the truth)
flux = 0.12
sig = 0.6
incl = 50.
PA = 70.
offx = -0.3
offy = -0.2
nominal_SB = (sig/rcenter)**0.7 * np.exp(-(rcenter/sig)**2.5)	# fullA distribution
int_SB = np.trapz(2.*np.pi*nominal_SB*rcenter, rcenter)		# a check on the total flux
nominal_SB *= flux / int_SB

muw = nominal_SB #Truth

In [ ]:
#Plot visibilities to compare data, truth, and uniform output
plt.plot(rho, D, '-k', label='Data')
plt.plot(rho, np.dot(X, nominal_SB), '-m', label= 'Truth')
plt.plot(rho, np.dot(X, wu), 'ob', label='Uniform prior')
plt.title('Visibilities')
plt.legend()

In [ ]:
#Plot SB to compare data, truth, and uniform output
plt.subplot(211)
plt.plot(rcenter, nominal_SB, '-k', label='Truth')
plt.plot(rcenter, wu, 'ob', label='Uniform')
plt.legend()
plt.title ('Surface Brightness vs Angle')
plt.subplot(212)
plt.plot(rcenter, nominal_SB - wu)
plt.title('Truth-model')
print "!!This tends to look better for smaller Ndata, suggesting there still may be an issue with wu calculation!!"

In [ ]:
#Calculate the GP covariance matrix (Cw) from the kernel (k), with mean muw
#The mean of the distribution with this prior is wgp, with variance Cgp
gpa = 1. #Hyperparameter amplitude
gpl = .05 #Hyperparameter lengthscale
Cw = calccovar(rcenter, gpa, gpl)
Cwinv = np.linalg.inv(calccovar(rcenter, gpa, gpl))

In [ ]:
plt.imshow(np.dot(Cw, Cwinv))
plt.colorbar()
#If it doesn't look like the identity matrix, something is wrong! Check nugget term in kernel.

In [ ]:
#Non-inverse method to getting wgp
Cgpinv = np.linalg.solve(Cu, np.eye(Nrings) + np.dot(Cu, Cwinv))
Cuinvwu = np.linalg.solve(Cu, wu)
wgp = np.linalg.solve(Cgpinv, Cuinvwu + np.dot(Cwinv, muw))

In [ ]:
plt.plot(rcenter, np.dot(Cu, Cuinvwu), 'c') #Cu.inv(Cu).wu
plt.plot(rcenter, wu, 'r') #wu
plt.title('These should look equal')


In [ ]:
#Plot visibilities
plt.plot(rho, D, '-k', label='Data')
plt.plot(rho, np.dot(X, nominal_SB), '-m', label= 'Truth')
plt.plot(rho, np.dot(X, wu), 'ob', label='Uniform prior')
plt.plot(rho, np.dot(X, wgp), 'og', label='GP prior')
plt.title('Visibilities')
plt.legend()

In [ ]:
#Plot SB
plt.subplot(211)
plt.plot(rcenter, nominal_SB, '-k', label='Truth')
plt.plot(rcenter, wu, 'ob', label='Uniform')
plt.plot(rcenter, wgp, 'og', label='GP')
plt.legend()
plt.title ('Surface Brightness vs Angle')
plt.subplot(212)
plt.plot(rcenter, nominal_SB - wu,'-b')
plt.plot(rcenter, nominal_SB - wgp, '-g')
plt.title('Truth-model')

#The Old Method That Uses Inverses

In [ ]:
#The regular inverse way of doing all of this:
Cuinv0 = np.linalg.inv(Cu)
wu0 = np.dot(Cuinv0, np.dot(np.dot(X.T, Sigmainv), D)) 
Cgp0 = np.linalg.inv(Cuinv0+Cwinv)
wgp0 = np.dot(Cgp0,(np.dot(Cuinv0, wu0) + np.dot(Cwinv, muw)))

In [ ]:
#Checking Cuinv0
plt.imshow(np.dot(Cu,Cuinv0))
plt.colorbar()
print 'Cuinv0 does not work well'

In [ ]:
#Compare wu and wu0
print 'Truth', nominal_SB
print 'With solve', wu
print 'With inv', wu0
plt.subplot(211)
plt.plot(rcenter, wu-wu0)
plt.title('Difference between w_u calc methods')
plt.ylabel('wu-wu0')
plt.subplot(212)
plt.plot(rcenter, wu-nominal_SB, ':k', label='Truth')
plt.ylabel('Comparison to truth')


In [ ]:
#Look at Cgp and Cgpinv
plt.imshow(np.dot(Cgp0, (Cuinv0+Cwinv)))
print 'This should look like the identity matrix, but it depends strongly on the hyperparameters.'

In [ ]:
np.linalg.cond(Cu)/1e16

In [ ]:
print np.amin(np.diag(Cu))
Cueps = Cu + np.eye(Nrings)*10.
np.linalg.cond(Cueps)/1e6
Cuepsinv = np.linalg.inv(Cueps)
plt.imshow(np.dot(Cueps, Cuepsinv))

In [ ]:
wueps = np.dot(Cuepsinv, np.dot(np.dot(X.T, Sigmainv), D)) 
Cgpeps = np.linalg.inv(Cuepsinv+Cwinv)
wgpeps = np.dot(Cgpeps,(np.dot(Cuepsinv, wueps) + np.dot(Cwinv, muw)))

In [ ]:
plt.imshow(np .dot(Cgpeps,Cuepsinv+Cwinv))
plt.colorbar()

In [ ]:
#Plot visibilities
plt.plot(rho, D, '-k', label='Data')
plt.plot(rho, np.dot(X, nominal_SB), '-m', label= 'Truth')
plt.plot(rho, np.dot(X, wu), 'ob', label='Uniform prior')
plt.plot(rho, np.dot(X, wueps), 'sc', label='Uniform prior')
#plt.plot(rho, np.dot(X, wgp), 'or', label='GP prior')
plt.plot(rho, np.dot(X, wgpeps), 'sg', label='GP prior')
plt.title('Visibilities')
plt.legend()

In [ ]:
plt.plot(rcenter, nominal_SB, '-k', label='Truth')
#plt.plot(rcenter, wu, 'ob', label='Uniform')
plt.plot(rcenter, wueps, 'sc', label='Uniform')
#plt.plot(rcenter, wgp, 'or', label='GP')
plt.plot(rcenter, wgpeps, 'sg', label='GP')
plt.legend()

In [ ]:
rcenter[2]-rcenter[1]

In [ ]:
plt.plot(rcenter, (nominal_SB-wgpeps)/nominal_SB,'o')
plt.ylabel("Fractional difference from truth")

In [ ]:
plt.subplot(311)
plt.title('Draws from Cw')
for draw in np.random.multivariate_normal(muw, Cw, size=20):
    plt.plot(rcenter, draw, color="0.5")

plt.plot(rcenter, nominal_SB-wueps, 'bo')

plt.subplot(312)       
plt.title('Draws from Cu')
for draw in np.random.multivariate_normal(wueps, Cueps, size=20):
    plt.plot(rcenter, draw, color="0.5")

#plt.plot(cb, result, color="b", label=r"$\hat{w}$")
#plt.plot(cb, wtilde, color="r", label=r"${w}~w/GP$")
#plt.plot(cb, nominal_SB, color="k", label="truth")


plt.subplot(313)
plt.title('Draws from Cgp')
this = np.random.multivariate_normal(wgpeps, Cgpeps, size=10000)

for draw in np.random.multivariate_normal(wgpeps, Cgpeps, size=20):
    plt.plot(rcenter, draw, color="0.5")
#plt.plot(cb, result, color="b", label=r"$\hat{w}$", marker = 'o', linestyle='-')
plt.plot(rcenter, wgpeps, color="r", label=r"${w}~w/GP$", marker = 'o', linestyle='-')
#plt.plot(cb, nominal_SB, color="k", label="truth")


# plt.legend(loc="upper right")

In [ ]:
2